# Wind Turbine Farm Management Assistant


## Objective

This notebook demonstrates the following:

- Utilizing Assistant tools such as the Code Interpreter and Function calling, this bot is capable of retrieving a CSV file that illustrates turbine wind speed, voltage, and the last maintenance date. It assists you in reviewing the file contents and aids in determining whether a specific turbine is in need of maintenance.

This tutorial uses the following Azure AI services:
- Access to Azure OpenAI Service - you can apply for access [here](https://aka.ms/oai/access)
- Azure OpenAI service - you can create it from instructions [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource)
- Azure OpenAI Studio - go to [https://oai.azure.com/](https://oai.azure.com/) to work with the Assistants API Playground
- A connection to the Azure OpenAI Service with a [Key and Endpoint](https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart)

Reference:
- Learn more about how to use Assistants with our [How-to guide on Assistants](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/assistant).
- [Assistants OpenAI Overview](https://platform.openai.com/docs/assistants/overview)


## Time

You should expect to spend 5-10 minutes running this sample.

## About this example

This sample demonstrates the creation of an Azure OpenAI Assistant named "Turbine Management Assistant" utilizing the Azure OpenAI API. The assistant functions as a management tool for wind turbine farms, providing data, charts, and actionable insights regarding the status and performance of each turbine within the farm. The notebook/script orchestrates a dialogue with the assistant, navigating through different inquiries and scenarios to demonstrate its capabilities.

### Data

This sample uses files from the folder [`data/`](./data/) in this repo. You can clone this repo or copy this folder to make sure you have access to these files when running the sample.


## Before you begin



### Installation

Install the following packages required to execute this notebook. 



In [ ]:
# Install the packages
%pip install -r ../requirements.txt

### Parameters

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv("../.env")

api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")
email_URI = os.getenv("EMAIL_URI")

should_cleanup: bool = False

## Run this Example

### Load the required libraries

In [ ]:
import io
from datetime import datetime
from pathlib import Path
from typing import Iterable
from PIL import Image

from openai import AzureOpenAI
from openai.types import FileObject
from openai.types.beta.threads import Message, TextContentBlock, ImageFileContentBlock

### Create an AzureOpenAI client

In [ ]:
client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint)

### Define the Assistant tools

In [ ]:
tools_list = [
    {"type": "code_interpreter"},
]

### Upload the Assistant file(s)

In [ ]:
DATA_FOLDER = "data/"


def upload_file(client: AzureOpenAI, path: str) -> FileObject:
    with Path(path).open("rb") as f:
        return client.files.create(file=f, purpose="assistants")


arr = os.listdir(DATA_FOLDER)
assistant_files = []
for file in arr:
    filePath = DATA_FOLDER + file
    assistant_files.append(upload_file(client, filePath))

file_ids = [file.id for file in assistant_files]

### Create an Assistant and a Thread

In [ ]:
assistant = client.beta.assistants.create(
    name="Portfolio Management Assistant",
    instructions="You are an assistant that can help manage wind turbine farm. "
    + "The turbines operating ranges are output voltages of 33kv-35kv and RPM of 15-25. Wind speed is measured in miles per hour."
    + "Maintenance should occur every 12 months. Greet the user by saying, 'Welcome Turbine Management Assistant.'",
    tools=tools_list,
    model=api_deployment_name,
    tool_resources={"code_interpreter": {"file_ids": file_ids}},
)

thread = client.beta.threads.create()

### Format and display the Assistant Messages for text and image

In [ ]:
def format_messages(messages: Iterable[Message]) -> None:
    message_list = []

    # Get all the messages till the last user message
    for message in messages:
        message_list.append(message)
        if message.role == "user":
            break

    # Reverse the messages to show the last user message first
    message_list.reverse()

    # Print the user or Assistant messages or images
    for message in message_list:
        for item in message.content:
            # Determine the content type
            if isinstance(item, TextContentBlock):
                print(f"{message.role}:\n{item.text.value}\n")
            elif isinstance(item, ImageFileContentBlock):
                # Retrieve image from file id
                response_content = client.files.content(item.image_file.file_id)
                data_in_bytes = response_content.read()
                # Convert bytes to image
                readable_buffer = io.BytesIO(data_in_bytes)
                image = Image.open(readable_buffer)
                # Resize image to fit in terminal
                width, height = image.size
                image = image.resize((width // 2, height // 2), Image.LANCZOS)
                # Display image
                image.show()

### Process the user messages

In [ ]:
def process_message(content: str) -> None:
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=content)

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="The current date and time is: " + datetime.now().strftime("%x %X") + ". ",
    )

    print("processing...")
    completed_run = client.beta.threads.runs.poll(run_id=run.id, thread_id=thread.id)

    # Check final status and handle accordingly
    if completed_run.status == "completed":
        print("Run completed successfully.")
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        format_messages(messages)
    elif completed_run.status == "failed":
        print("Run failed.")
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        format_messages(messages)
    else:
        print(f"Unexpected run status: {completed_run.status}")

###  Have a conversation with the Assistant

In [ ]:
process_message("What is the status of turbine 1001 and 1003?")

In [ ]:
process_message("Generate a chart of the normal operating ranges?")

In [ ]:
process_message(
    "What turbines are operating outside normal ranges?\n\n"
    + 'Sample: [{"turbine":"","voltage":30,"RPM":15,"reason":"Voltage outside of normal range."},\n'
    + '{"turbine":"","voltage":30,"rpm":10,"reason":"RPM and Voltage outside of normal ranges."},\n'
    + '{"turbine":"","voltage":33,"rpm":5,"reason":"RPM outside of normal range."}]\n\n'
    + 'Output format: [{"turbine":"","voltage":0,"RPM":0,"reason":""}]\n\n'
    + "Output in JSON format only."
)

In [ ]:
process_message("What turbines need maintenance?")

## Cleaning up


In [ ]:
if should_cleanup:
    client.beta.assistants.delete(assistant.id)
    client.beta.threads.delete(thread.id)
    for file in assistant_files:
        client.files.delete(file.id)